## Rasa nlu

## Clone git repo

In [1]:
BRANCH_NAME='master'
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 3361 (delta 20), reused 112 (delta 14), pack-reused 3236
Receiving objects: 100% (3361/3361), 12.09 MiB | 6.05 MiB/s, done.
Resolving deltas: 100% (1271/1271), done.


## Create requirements.txt

In [2]:
%%writefile requirements.txt
kfp
minio
boto3
rasa
nest_asyncio
lxml
bs4
rasa_sdk

Writing requirements.txt


## Install libraries

In [3]:
!pip install -r requirements.txt --user

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import libraries

In [1]:
import kfp
import os
import boto3
from kubernetes import client, config
from datetime import datetime

import asyncio
import nest_asyncio
from minio import Minio
from rasa.jupyter import chat
from typing import Dict, Text
from rasa.run import create_agent
from botocore.client import Config
from rasa.core.lock_store import LockStore
from botocore.exceptions import ClientError
from rasa.core.tracker_store import TrackerStore
from rasa.core.utils import AvailableEndpoints
from rasa.core.agent import Agent
from rasa.core.brokers.broker import EventBroker
from rasa.cli.utils import print_success, print_error

nest_asyncio.apply()


config.load_incluster_config()
api_client = client.CoreV1Api()

## Component files Declarations

In [2]:
path='cisco-kubeflow-starter-pack/apps/healthcare/covid-forecasting/onprem/pipelines/'
component_root_train = path+'components/rasa-train/'
component_root_evaluate = path+'components/rasa-evaluate/'
component_root_upload = path+'components/rasa-upload/'

## Load components from respective .YAML config files

In [3]:
rasa_train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
rasa_evaluate_op = kfp.components.load_component_from_file(os.path.join(component_root_evaluate, 'component.yaml'))
rasa_upload_op = kfp.components.load_component_from_file(os.path.join(component_root_upload, 'component.yaml'))

## Define Volume and Volume Mounts

In [4]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

## Define pipeline function

In [5]:
def faq_pipeline(
    base="/ml",
    models_dir="models",
    domain_yml="domain.yml",
    config_yml="config.yml",
    nlu_core_md="data",
    model_name="rasa_model",
    eval_stories_md = "tests/conversation_tests.md",
    results_dir="results",
    output_dir="/mnt",
    minio_bucket='rasa'
    ):
    
    models_path = str(output_dir) + "/" + str(models_dir)
    resutls_path = str(models_path) + "/" + str(results_dir)
    
    train = rasa_train_op(out_path=models_path)
    train.add_volume(nfs_volume)
    train.add_volume_mount(nfs_volume_mount)
    
    eval = rasa_evaluate_op(out_path=resutls_path)
    eval.add_volume(nfs_volume)
    eval.add_volume_mount(nfs_volume_mount)
    eval.after(train)
    
    upload = rasa_upload_op(minio_bucket=minio_bucket)
    upload.add_volume(nfs_volume)
    upload.add_volume_mount(nfs_volume_mount)
    upload.after(eval)

In [6]:
try:
    import kfp.compiler as compiler
    compiler.Compiler().compile(faq_pipeline, 'faq_pipeline.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

## Create kubeflow pipeline experiment

In [7]:
kf_client = kfp.Client()
rasa_experiment = kf_client.create_experiment(name='COVID-FAQ')

## Create kubeflow pipeline run

In [9]:
timestamp = datetime.now().strftime("%d_%m_%y_%H_%M_%S")
minio_bucket='rasa'
run = kf_client.run_pipeline(rasa_experiment.id, 'covid-faq-pipeline-'+timestamp, pipeline_package_path='faq_pipeline.tar.gz', params={"minio_bucket":minio_bucket})

Note: wait for experiment succeeded

## Download model from Minio

### Get minio-service cluster IP to download rasa model

In [10]:
try:
    minio_service_endpoint = api_client.read_namespaced_service(name='minio-service', namespace='kubeflow').spec.cluster_ip
    minio_service_enpoint_port=api_client.read_namespaced_service(name='minio-service', namespace='kubeflow').spec.ports[0].port
except ApiException as e:
            if e.status == 403:
                logging.warning(f"The service account doesn't have sufficient privileges "
                      f"to get the kubeflow minio-service. "
                      f"You will have to manually enter the minio cluster-ip. "
                      f"To make this function work ask someone with cluster "
                      f"priveleges to create an appropriate "
                      f"clusterrolebinding by running a command.\n"
                      f"kubectl create --namespace=kubeflow rolebinding "
                       "--clusterrole=kubeflow-view "
                       "--serviceaccount=${NAMESPACE}:default-editor "
                       "${NAMESPACE}-minio-view")
                logging.error("API access denied with reason: {e.reason}")

minio_endpoint = "http://"+ minio_service_endpoint  + ":%s"%minio_service_enpoint_port
print("minio endopoint : ", minio_endpoint)

minio endopoint :  http://10.39.248.229:9000


In [13]:
boto_client = boto3.client('s3',
                      endpoint_url=minio_endpoint,
                      aws_access_key_id="minio",
                      aws_secret_access_key="minio123",
                      config=Config(signature_version='s3v4'),
                      region_name="us-east-1",
                      use_ssl=False)

## Download model from minio bucket
filename='models.tar.gz'
boto_client.download_file(Bucket=minio_bucket, Key='models.tar.gz', Filename=filename)
print("Rasa model downloaded")

Rasa model downloaded


## untar model

In [14]:
!tar -xvf $filename

model/
model/rasa_model.tar.gz
model/results/
model/results/CRFEntityExtractor_errors.json
model/results/CRFEntityExtractor_report.json
model/results/DIETClassifier_errors.json
model/results/DIETClassifier_report.json
model/results/confmat.png
model/results/failed_stories.md
model/results/hist.png
model/results/intent_errors.json
model/results/intent_report.json
model/results/response_selection_confmat.png
model/results/response_selection_report.json
model/results/story_confmat.pdf


## Chat with your assistant

In [15]:
def _display_bot_response(response: Dict):
    from IPython.display import Image, display  # pytype: disable=import-error

    for response_type, value in response.items():
        if response_type == "text":
            print_success(value)

        if response_type == "image":
            image = Image(url=value)
            display(image)
            
def create_agent(model: Text, endpoints: Text = None) -> "Agent":
    _endpoints = AvailableEndpoints.read_endpoints(endpoints)
    _broker = EventBroker.create(_endpoints.event_broker)
    _tracker_store = TrackerStore.create(_endpoints.tracker_store, event_broker=_broker)
    _lock_store = LockStore.create(_endpoints.lock_store)

    return Agent.load(
        model,
        generator=_endpoints.nlg,
        tracker_store=_tracker_store,
        lock_store=_lock_store,
        action_endpoint=_endpoints.action,
    )
agent = create_agent("model/rasa_model.tar.gz", endpoints="cisco-kubeflow-starter-pack/apps/healthcare/covid-forecasting/onprem/pipelines/rasa/endpoints.yml")

/home/jovyan/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Instructions for updating:
Use tf.identity instead.


In [16]:
print("Your bot is ready to talk! Type your messages here or send '/stop'.")
loop = asyncio.get_event_loop()
while True:
    message = input()
    if message == "/stop":
        break
    responses = loop.run_until_complete(agent.handle_text(message))
    for response in responses:
        _display_bot_response(response)

Your bot is ready to talk! Type your messages here or send '/stop'.
Hi
Hi!
Select the source:
 - WHO(World Health Organization)
 - MOHFW(Ministry of Health and Family Welfare of India)
 - US(United Nations)
 - CDC(Centers for Disease Control and Prevention)
 - DEFAULT
WHO
You are selected:  who
What is coronavirus?
Coronaviruses are a large family of viruses which may cause illness in animals or humans.  In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19.
What is covid?
COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019. COVID-19 is now a pandemic affecting many countries globally.
How many peoples are infect

## Delete bucket from minio server

In [17]:
boto_client.delete_object(Bucket=minio_bucket, Key=filename)

{'ResponseMetadata': {'RequestId': '16168BEDC65EBDA9',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'server': 'Minio/RELEASE.2018-02-09T22-40-05Z (linux; amd64)',
   'vary': 'Origin',
   'x-amz-request-id': '16168BEDC65EBDA9',
   'date': 'Mon, 08 Jun 2020 10:49:31 GMT'},
  'RetryAttempts': 0}}

In [18]:
boto_client.delete_bucket(Bucket=minio_bucket)

{'ResponseMetadata': {'RequestId': '16168BEE2512DF2D',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'server': 'Minio/RELEASE.2018-02-09T22-40-05Z (linux; amd64)',
   'vary': 'Origin',
   'x-amz-request-id': '16168BEE2512DF2D',
   'date': 'Mon, 08 Jun 2020 10:49:33 GMT'},
  'RetryAttempts': 0}}

In [19]:
!rm -rf $filename model/ requirements.txt